In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-02 02:23:33.892743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 02:23:34.954919: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')

imdb_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model.to(device)
imdb_pipeline = pipeline('sentiment-analysis', model=imdb_model, tokenizer=imdb_tokenizer)
imdb_pipeline.device = next(imdb_model.parameters()).device

imdb_model_directory = "../../../models/bert-uncased_maskedlm_imdb_july31_chk3"
finetuned_imdb_maskedlm = BertForMaskedLM.from_pretrained(imdb_model_directory)
finetuned_imdb_maskedlm.to(device)
imdb_fill_mask = pipeline("fill-mask", model=finetuned_imdb_maskedlm, tokenizer=imdb_tokenizer)
imdb_fill_mask.device = next(imdb_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

dataset_val = input('Enter the number of samples to run on (100 or 776): ')

if dataset_val == '100':
    loaded_imdb_100 = Dataset.load_from_disk('../data/filtered_imdb_clean_100')
    imdb_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_100))
    dataset = imdb_100
    dataset_name = 'imdb100'
elif dataset_val =='776':
    loaded_imdb_776 = Dataset.load_from_disk('../data/filtered_imdb_clean_776')
    imdb_776 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_776))
    dataset = imdb_776
    dataset_name = 'imdb776'
else:
    raise ValueError('Number of samples not supported')
    
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
if defense == "default":
    imdb_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(imdb_model, imdb_tokenizer)
elif defense == "logit":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(imdb_wrapper)

cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))
# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.7 theshold for imdb Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.7, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:1
Enter the number of samples to run on (100 or 776): 100
Specify a defense type among "default", "logit", "maj_log", "one_hot": maj_log
using num_voter = 11 and mask_pct = 0.3 with dataset = imdb100...
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 12
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  12
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
  


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-02 02:29:01.912268: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-02 02:29:03.781815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]

  1%|          | 1/100 [13:33<22:21:35,


[Succeeded / Failed / Skipped / Total] 4 / 17 / 4 / 25:  26%|██▌       | 26/100 [4:28:39<12:44:39, 620.00s/it]
[Succeeded / Failed / Skipped / Total] 4 / 18 / 4 / 26:  26%|██▌       | 26/100 [4:28:40<12:44:40, 620.00s/it]
[Succeeded / Failed / Skipped / Total] 4 / 18 / 4 / 26:  27%|██▋       | 27/100 [4:34:14<12:21:28, 609.43s/it]
[Succeeded / Failed / Skipped / Total] 4 / 19 / 4 / 27:  27%|██▋       | 27/100 [4:34:14<12:21:28, 609.43s/it]
[Succeeded / Failed / Skipped / Total] 4 / 19 / 4 / 27:  28%|██▊       | 28/100 [4:49:09<12:23:32, 619.62s/it]
[Succeeded / Failed / Skipped / Total] 4 / 20 / 4 / 28:  28%|██▊       | 28/100 [4:49:09<12:23:33, 619.63s/it]
[Succeeded / Failed / Skipped / Total] 4 / 20 / 4 / 28:  29%|██▉       | 29/100 [4:53:55<11:59:36, 608.12s/it]
[Succeeded / Failed / Skipped / Total] 5 / 20 / 4 / 29:  29%|██▉       | 29/100 [4:53:55<11:59:36, 608.12s/it]
[Succeeded / Failed / Skipped / Total] 5 / 20 / 4 / 29:  30%|███       | 30/100 [4:55:53<11:30:24, 591.78s/it]



[Succeeded / Failed / Skipped / Total] 9 / 36 / 5 / 50:  51%|█████     | 51/100 [10:33:26<10:08:35, 745.22s/it]
[Succeeded / Failed / Skipped / Total] 9 / 37 / 5 / 51:  51%|█████     | 51/100 [10:33:26<10:08:35, 745.22s/it]
[Succeeded / Failed / Skipped / Total] 9 / 37 / 5 / 51:  52%|█████▏    | 52/100 [10:49:08<9:59:12, 749.02s/it] 
[Succeeded / Failed / Skipped / Total] 9 / 38 / 5 / 52:  52%|█████▏    | 52/100 [10:49:08<9:59:12, 749.02s/it]
[Succeeded / Failed / Skipped / Total] 9 / 38 / 5 / 52:  53%|█████▎    | 53/100 [11:06:10<9:50:45, 754.15s/it]
[Succeeded / Failed / Skipped / Total] 9 / 39 / 5 / 53:  53%|█████▎    | 53/100 [11:06:10<9:50:45, 754.15s/it]
[Succeeded / Failed / Skipped / Total] 9 / 39 / 5 / 53:  54%|█████▍    | 54/100 [11:20:21<9:39:33, 755.95s/it]
[Succeeded / Failed / Skipped / Total] 9 / 40 / 5 / 54:  54%|█████▍    | 54/100 [11:20:21<9:39:33, 755.95s/it]
[Succeeded / Failed / Skipped / Total] 9 / 40 / 5 / 54:  55%|█████▌    | 55/100 [11:24:22<9:19:56, 746.60s/i


[Succeeded / Failed / Skipped / Total] 11 / 58 / 6 / 75:  76%|███████▌  | 76/100 [17:05:38<5:23:53, 809.72s/it]
[Succeeded / Failed / Skipped / Total] 11 / 59 / 6 / 76:  76%|███████▌  | 76/100 [17:05:38<5:23:53, 809.72s/it]
[Succeeded / Failed / Skipped / Total] 11 / 59 / 6 / 76:  77%|███████▋  | 77/100 [17:22:14<5:11:19, 812.14s/it]
[Succeeded / Failed / Skipped / Total] 11 / 60 / 6 / 77:  77%|███████▋  | 77/100 [17:22:14<5:11:19, 812.14s/it]
[Succeeded / Failed / Skipped / Total] 11 / 60 / 6 / 77:  78%|███████▊  | 78/100 [18:28:47<5:12:44, 852.92s/it]
[Succeeded / Failed / Skipped / Total] 11 / 61 / 6 / 78:  78%|███████▊  | 78/100 [18:28:47<5:12:44, 852.92s/it]
[Succeeded / Failed / Skipped / Total] 11 / 61 / 6 / 78:  79%|███████▉  | 79/100 [18:51:40<5:00:49, 859.50s/it]
[Succeeded / Failed / Skipped / Total] 11 / 62 / 6 / 79:  79%|███████▉  | 79/100 [18:51:40<5:00:49, 859.50s/it]
[Succeeded / Failed / Skipped / Total] 11 / 62 / 6 / 79:  80%|████████  | 80/100 [18:55:22<4:43:50, 851

[Succeeded / Failed / Skipped / Total] 16 / 75 / 9 / 100: 100%|██████████| 100/100 [23:26:11<00:00, 843.72s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 75     |
| Number of skipped attacks:    | 9      |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 75.0%  |
| Attack success rate:          | 17.58% |
| Average perturbed word %:     | 2.68%  |
| Average num. words per input: | 159.1  |
| Avg num queries:              | 752.98 |
+-------------------------------+--------+
The above are results for TextFoolerJin2019_imdb100_candsize12_mp0.3_nv11_maj_log.
